### Atividade Busca Semântica

Nesta atividade você deve aplicar os conhecimentos sobre word embeddings e tokenização para criar um mecanismos de busca semântica. Será disponibilizado um conjunto de dados que possui perguntas médicas, em inglês. Esse conjunto de dados possui as perguntas e também as respostas.

Você deve gerar os vetores das perguntas do conjunto de dados, e permitir que o "usuário" envie a sua pergunta. Você também deve gerar o vetor da pergunta do usuário e com isso buscar a resposta ideal para o usuário. **A resposta ideal é aquela onde o vetor da pergunta do usuário é mais similar ao vetor da pergunta do conjunto de dados** Consulte o notebook "nlp2.ipynb" para verificar como realizamos esse processo

Portanto, no seu script deve ser possível escrever um texto "pergunta" e deve ser retornado a resposta adequada, isto é, a resposta associada a pergunta mais similar no conjunto de dados.

Para isso utilize o pandas e os packages do huggingface

In [2]:
### Download do conjunto de dados

import kagglehub

# Download latest version
path = kagglehub.dataset_download("pythonafroz/medquad-medical-question-answer-for-ai-research")

print("Path to dataset files:", path)

/workspaces/aps/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 4.95M/4.95M [00:00<00:00, 134MB/s]

Extracting files...
Path to dataset files: /home/codespace/.cache/kagglehub/datasets/pythonafroz/medquad-medical-question-answer-for-ai-research/versions/1


In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers.tokenization_utils_base import BatchEncoding

In [4]:
## Amostra do conjunto

df = pd.read_csv(f"{path}/medquad.csv")
df_amostra = df.sample(5000)

In [5]:
# Modelo para a língua inglesa

nome_modelo = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)
model = AutoModel.from_pretrained(nome_modelo)

2025-09-17 01:04:56.150761: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
## Funções para: 1) Obter os tokens; 2) Obter os embeddings

def get_tokens(pergunta: str) -> str:
    return tokenizer(pergunta, return_tensors="pt")

def get_vetores(tokens_pergunta: BatchEncoding) -> Tensor:

    with torch.no_grad():
        outputs = model(**tokens_pergunta)
        embeddings = outputs.last_hidden_state

    return embeddings

In [7]:
## Novas colunas para os tokens e para os embeddings

df_amostra['tokens'] = df_amostra['question'].apply(lambda x: get_tokens(x))
df_amostra['vetores'] = df_amostra['tokens'].apply(lambda x: get_vetores(x))

In [8]:
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Aplica a função de média para obter os vetores de sentença
df_amostra['vetores_medios'] = df_amostra.apply(lambda row: average_pool(row['vetores'], row['tokens']['attention_mask']), axis=1)

# A pergunta do usuário
pergunta_usuario = "What are the symptoms of a heart attack?"

# Gera o vetor para a pergunta do usuário
tokens_usuario = get_tokens(pergunta_usuario)
vetores_usuario = get_vetores(tokens_usuario)
vetor_medio_usuario = average_pool(vetores_usuario, tokens_usuario['attention_mask'])

# Calcula a similaridade de cossenos entre a pergunta do usuário e todas as perguntas do conjunto de dados
print("Calculando a similaridade...")
similarities = []
for index, row in df_amostra.iterrows():
    vetor_pergunta_df = row['vetores_medios']
    similarity = F.cosine_similarity(vetor_medio_usuario, vetor_pergunta_df)
    similarities.append(similarity.item())

df_amostra['similaridade'] = similarities
print("Cálculo da similaridade concluído.")

# Encontra a pergunta mais similar com base na maior pontuação de similaridade
indice_mais_similar = df_amostra['similaridade'].idxmax()
pergunta_mais_similar = df_amostra.loc[indice_mais_similar, 'question']
resposta_correspondente = df_amostra.loc[indice_mais_similar, 'answer']

# Exibe o resultado final
print("\n" + "="*50)
print(f"Pergunta do Usuário: {pergunta_usuario}")
print("-" * 30)
print(f"Pergunta Mais Similar Encontrada: {pergunta_mais_similar}")
print("-" * 30)
print(f"Resposta: {resposta_correspondente}")
print("="*50)


Calculando a similaridade...
Cálculo da similaridade concluído.

Pergunta do Usuário: What are the symptoms of a heart attack?
------------------------------
Pergunta Mais Similar Encontrada: What are the symptoms of Heart Attack ?
------------------------------
Resposta: Symptoms Can Vary Not all heart attacks begin with the sudden, crushing chest pain that often is shown on TV or in the movies. The warning signs and symptoms of a heart attack aren't the same for everyone. Many heart attacks start slowly as mild pain or discomfort. Some people don't have symptoms at all. Heart attacks that occur without any symptoms or very mild symptoms are called silent heart attacks. However, some people may have a pattern of symptoms that recur. The more signs and symptoms you have, the more likely it is that you're having a heart attack If you have a second heart attack, your symptoms may not be the same as the first heart attack. Here are common signs and symptoms of a heart attack. Chest Pain o